In [17]:
import sys
import os

sys.path.append('scripts')

import re
import json
import argparse
import itertools
from pathlib import Path
import collections
from concurrent.futures import ProcessPoolExecutor

import numpy as np
import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm
from matplotlib.patches import FancyBboxPatch
from matplotlib.patches import PathPatch
from matplotlib.path import get_path_collection_extents
import seaborn as sns

from rich import print, pretty
from typing import  Iterable
import pretty_errors
from catppuccin import PALETTE

print(os.getcwd())

# import .scripts.ldj
from ldj import ldj
from utils import *

pretty.install()

RESULTS_DIRS = [
    Path('./experiments/varying-network-connectivity-lm-3-th-13-sd-2.5'),
    Path('./experiments/varying-network-connectivity-gbpplanner'),
]

for RESULTS_DIR in RESULTS_DIRS:
    assert RESULTS_DIR.is_dir() and RESULTS_DIR.exists()

# RESULTS_DIR = Path('./experiments/circle-experiment-lm-3-th-5')
# assert RESULTS_DIR.is_dir() and RESULTS_DIR.exists()

flavor = PALETTE.latte.colors
# num-robots-10-seed-0.json
RE = re.compile(r"comms-radius-(\d+)-seed-(\d+).json")


/home/jens/repos/master/gbp-rs

In [18]:
# use LaTeX for text with matplotlib
sns.set_style("darkgrid")
# set background color of sns darkgrid to flavor.base.hex
plt.rcParams['axes.facecolor'] = flavor.base.hex
# set font color to flavor.text.hex
plt.rcParams['text.color'] = flavor.text.hex

font_dirs = ["./scripts/fonts/"]
# go through all fonts in the font directory and add them
for font_dir in font_dirs:
    for font in os.listdir(font_dir):
        fm.fontManager.addfont(f"{font_dir}/{font}")

prop_jbm = fm.FontProperties(fname='./scripts/fonts/JetBrainsMonoNerdFontMono-Regular.ttf')
prop = fm.FontProperties(fname='./scripts/fonts/STIXTwoText-VariableFont_wght.ttf')

plt.rcParams.update({
    # "text.usetex": True,
    "font.family": prop.get_name(),
    # "font.family": "stix",
    # "font.sans-serif": prop.get_name(),
    "mathtext.fontset": "stix",
    # "text.latex.preamble": r"\usepackage{fontenc}\usepackage{fontspec}\setmainfont{JetBrainsMonoNerdFontMono-Regular}",
})

print(prop.get_name())

colors = [(flavor.lavender.hex, 1.0), (flavor.yellow.hex, 0.3), (flavor.peach.hex, 0.3)]

STIX Two Text

In [44]:

def flatten(lst: Iterable) -> list:
    return list(itertools.chain.from_iterable(lst))

def process_file_(file):
    # print(f"Processing {file}")
    match = RE.match(file.name)
    assert match is not None
    comms_radius = int(match.group(1))
    seed = int(match.group(2))

    with open(file, 'r') as file:
        data = json.load(file)

    distance_travelled_of_each_robot: list[float] = []
    ldj_of_each_robot: list[float] = []
    time_to_completion_of_each_robot: list[float] = []

    for _, robot_data in data['robots'].items():
        positions = np.array(robot_data['positions'])
        
        distance_travelled = np.sum(np.linalg.norm(np.diff(positions, axis=0), axis=1))
        distance_travelled_of_each_robot.append(distance_travelled)
        mission = robot_data['mission']
        
        t_start: float = mission['started_at']
        t_final: float = mission['finished_at'] if mission['finished_at'] else mission['duration'] + t_start
        time_to_completion_of_each_robot.append(t_final - t_start)

        # timestamps: np.ndarray = np.array([measurement['timestamp'] for measurement in robot_data['velocities']])
        timestamps: np.ndarray = np.array([t_start + 0.1 * i for i in range(len(robot_data['velocities']))])
        velocities3d_bevy: np.ndarray = np.array([measurement['velocity'] if isinstance(measurement, dict) else measurement for measurement in robot_data['velocities']])
        # if velocities3d_bevy.shape[1] != 3: print(velocities3d_bevy.shape)
        velocities = velocities3d_bevy[:, [0, 2]] if velocities3d_bevy.shape[1] == 3 else velocities3d_bevy

        # metric = ldj(velocities, timestamps)
        metric = ldj(velocities, timestamps)
        ldj_of_each_robot.append(metric)

    # makespan: float = data['makespan']
    makespan: float = max(time_to_completion_of_each_robot)
    # print(f"{makespan=}")
    return comms_radius, distance_travelled_of_each_robot, makespan, ldj_of_each_robot

In [45]:
[0.1 * i for i in range(10)]

[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9]

In [46]:

data = []
for RESULTS_DIR in RESULTS_DIRS:
    # with ProcessPoolExecutor() as executor:
    #     results = executor.map(process_file_, RESULTS_DIR.glob('*.json'))

    print(f"Processing {RESULTS_DIR=}")

    results = [process_file_(file) for file in RESULTS_DIR.glob('*.json')]

    # Aggregate results in a single-threaded manner to avoid data
    aggregated_data_distance_travelled: dict[int, list[float]] = collections.defaultdict(list)
    aggregated_data_makespan: dict[int, list[float]] = collections.defaultdict(list)
    aggregated_data_ldj: dict[int, list[float]] = collections.defaultdict(list)

    for comms_radius, distance_travelled_for_each_robot, makespan, ldj_for_each_robot in results:
        # print(f"{comms_radius=} {makespan=}")
        aggregated_data_distance_travelled[comms_radius].extend(distance_travelled_for_each_robot)
        aggregated_data_makespan[comms_radius].append(makespan)
        aggregated_data_ldj[comms_radius].extend(ldj_for_each_robot)

    data_distance = [aggregated_data_distance_travelled[key] for key in sorted(aggregated_data_distance_travelled.keys())]
    labels_distance = sorted(aggregated_data_distance_travelled.keys())

    data_distance_dict = dict(zip(labels_distance, data_distance))

    data.append(
        {
            'distance': data_distance_dict,
            # 'labels_distance': labels_distance,
            'makespan': aggregated_data_makespan,
            'ldj': aggregated_data_ldj
        }
    )


Processing RESULTS_DIR=PosixPath('experiments/varying-network-connectivity-lm-3-th-13-sd-2.5')

Processing RESULTS_DIR=PosixPath('experiments/varying-network-connectivity-gbpplanner')

In [47]:
data


[
    {
        'distance': {
            20: [
                222.49901027559036,
                234.6549748032037,
                239.32791511758487,
                261.00184913485566,
                292.9038024337787,
                254.79939806828762,
                239.44698862673266,
                251.76605376923146,
                216.13657164522235,
                218.10522973080393,
                211.36560261559262,
                217.75643406085905,
                269.1718608298344,
                221.96276201169508,
                227.2161075259396,
                221.5589143437578,
                261.59762546856854,
                253.20457271841877,
                238.10608413623993,
                277.11688168986143,
                220.15291684092875,
                259.16763738500106,
                212.04866719359143,
                217.43739142562373,
                264.8037520521517,
                244.92756422215896,
                212.0

In [48]:
# for each comms radius, [20, 40, 60, 80], calculate mean makespan, distance travelled, and ldj

for i, d in enumerate(data):

    makespan_means = dict([(key, np.mean(d['makespan'][key])) for key in sorted(d['makespan'].keys())])
    distance_means = dict([(key, np.mean(d['distance'][key])) for key in sorted(d['distance'].keys())])
    ldj_means = dict([(key, np.mean(d['ldj'][key])) for key in sorted(d['ldj'].keys())])

    print(f"{RESULTS_DIRS[i].name}")
    print(f"{makespan_means=}")
    print(f"{distance_means=}")
    print(f"{ldj_means=}")
    

varying-network-connectivity-lm-3-th-13-sd-2.5

makespan_means={20: 44.879999999999995, 40: 63.480000000000004, 60: 64.22, 80: 63.6}

distance_means={20: 236.89510927868324, 40: 285.9908325030257, 60: 295.2933289314799, 80: 283.24733969761115}

ldj_means={20: -14.479386124124119, 40: -15.966531007631177, 60: -16.334361227393277, 80: -16.105878378955726}

varying-network-connectivity-gbpplanner

makespan_means={20: 199.5, 40: 260.8999938964844, 60: 443.3000183105469, 80: 379.0}

distance_means={20: 519.4496599236194, 40: 751.0645575497416, 60: 1551.7318792183992, 80: 1069.3622512026793}

ldj_means={20: -22.93817249057638, 40: -23.742672523380392, 60: -25.33701200919546, 80: -24.86977648122963}